In [278]:
from random import *
from Crypto.Util.number import getPrime, isPrime
from math import gcd
from math import log2
from Crypto.Hash import SHA3_256
from math import floor
m="allo"
h = SHA3_256.new(bytes(m, "utf-8")).hexdigest()

In [279]:
a1=57896044618658097711785492504343953926634992332820282019728791901641727051837
b1=398341948620716521344
q1 = 2**252 + 27742317777372353535851937790883648493
G1=(2, 2587177637973221124604506650587198648324617568965701997790174145338249409797)
p1=(2**255-19)


EC25519=[p1,a1,b1,G1,q1]

In [3]:
EC=[19,1,1,[16,16],21]
pow(5,17,77)


3

In [281]:
def xgcd(a,b): #C'est une fonction pour un autre dm qui trouve les coeff de bezout avec THM Euclide etendu
  x,y=1,0
  u,v=0,1
  while b != 0:
   q,r=a//b,a%b
   m,n=x-u*q,y-v*q #On fait l'opération L1-q*L2 (1)
   a,b=b,r #On fait un décalage car la technique le veut. Le quotient devant le numérateur et le reste devient le quotient 
   x,y=u,v #Maintenant comme on fait L2-L3 les coefficients ne sont plus les mêmes pour la ligne de calcul(1), 
   u,v=m,n #donc on va echanger.
  reste=a
  return [reste,x,y] #a est le reste b 

In [282]:
def ec_zero():
    return "ZERO"
    
def ec_is_zero(P):
    if P=="ZERO":
        return True
    else :
        return False
    
ec_is_zero(ec_zero())

True

In [283]:
def ec_opp(EC,P):
    p=EC[0]
    if ec_is_zero(P)==True:
        return ec_zero()
    else:
        L=P.copy()
        L[1]=-L[1]%p #Question : operation de groupe, comment faire pour distinguer dans EC l'opération de groupe.
    return L

A =[1,10]

ec_opp(EC,A)




[1, 9]

In [284]:
def ec_double(EC,P): #Fonction bonne
    p=EC[0]
    if P[1]%p==-P[1]%p:
        return "ZERO"
    
    elif P=="ZERO":
        return "ZERO"
    
    else:
        u=(3*(P[0])**2+EC[1])*(pow(P[1]*2,-1,p))  
        x3=(u**2-2*P[0])
        y3=-P[1]+u*(P[0]-x3)
        return [x3%p,y3%p]




In [285]:
#EC=[23, 1, 2, (2, 14), 12]   
#ec_double(EC,[1,0])
EC=[19,1,1,[16,16],21]
ec_double(EC,[5,6])


[9, 13]

In [286]:
def ec_add(EC,P,Q) :
    p=EC[0]
    if P==Q :
        return ec_double(EC,P)
    elif P==ec_zero() and Q!=ec_zero():
        return Q
    elif P!=ec_zero() and Q==ec_zero():
        return P
    elif P==ec_zero() and Q==ec_zero():
        return ec_zero()
    elif P[0] % p == Q[0] % p and P[1] % p == (-Q[1]) % p:
        return "ZERO"
    
    else:
        if xgcd(Q[0]-P[0],p)[0]!=1:
            print("Le reste n'est pas égale à 1 mais à :",xgcd(Q[0]-P[0],p)[0])
        u=(Q[1]-P[1])*pow(Q[0]-P[0],-1,p)
        v=(P[1]-u*P[0])
        x3=(u**2-P[0]-Q[0])
        y3=(-P[1]+u*(P[0]-x3))
        
        return [x3%p,y3%p]
    

ec_add(EC,[14,2],[0,1])

[2, 7]

In [287]:
def ec_fast_mult(EC, P, m):
    p=EC[0]
    P=EC[-2]
    nombre_binaire = str(bin(m)[2:])
    m_liste_binaire = [int(chiffre) for chiffre in nombre_binaire]
    liste_binaire=m_liste_binaire[::-1]
    R=ec_zero()
    Q=P
    L=[]
    for i in range(0,len(liste_binaire)):
        if liste_binaire[i]==1:
            R=ec_add(EC,R,Q)
            
        Q=ec_add(EC,Q,Q)
        L.append(Q)

    return R

In [288]:
EC=[19,1,1,[16,16],21]

ec_fast_mult(EC,[16,16],5)

[9, 6]

In [289]:
def inverse_liste(L):
    E=[]
    for i in range (0,len(L)):
        E.append(L[-i])
    return E


In [290]:
from Crypto.Hash import SHA3_256

#Question 6 
chaine_test=str('Cryptographie à clé publique')
h3 = SHA3_256.new(bytes(chaine_test, "utf-8")).hexdigest()
print(h3=="2175c8e979e092c9dc9010e1c06ea490dff52f1bf5d9df8fd2b7f35b2ae58917")



True


In [291]:
#Question 7
h3_int=int(h3,16)
print(h3_int==15134431753598964768903003036524780383794702900043419402360626898381641713943)

True


In [292]:
# pour ouvrir le fichier et stocker l’objet correspondant dans la variable f :
OuvrirTestFile = open("testfile.txt", 'r')
# pour stocker dans une chaine de caracteres s le contenu du fichier associe a f
text_testfile = OuvrirTestFile.read()
# pour fermer le fichier (ne pas oublier)

hache_text=SHA3_256.new(bytes(text_testfile, "utf-8")).hexdigest()

hache_nombre=int(hache_text,16)%1000003
print(hache_nombre==943673)

True


In [293]:
#Note : je dois changer des trucs parce que c'est DSA pas ECDSA
def ecdsa_keygen(EC) :
    G,q=EC[-2:]
    p=EC[0]
    a=randint(1,q-1)

    A=ec_fast_mult(EC,G,a)

    return[A,a] #A:cle publique et a:cle secrete 




In [294]:
def xgcd(a,b): #C'est une fonction pour un autre dm qui trouve les coeff de bezout avec THM Euclide etendu
  x,y=1,0
  u,v=0,1
  while b != 0:
   q,r=a//b,a%b
   m,n=x-u*q,y-v*q #On fait l'opération L1-q*L2 (1)
   a,b=b,r #On fait un décalage car la technique le veut. Le quotient devant le numérateur et le reste devient le quotient 
   x,y=u,v #Maintenant comme on fait L2-L3 les coefficients ne sont plus les mêmes pour la ligne de calcul(1), 
   u,v=m,n #donc on va echanger.
  reste=a
  return [reste,x,y] #a est le reste b 

In [295]:
import random 
from Crypto.Hash import SHA3_256

def ecdsa_sign(EC,m,sk):
    p,q,G=EC[0],EC[-1],EC[-2]
    h = SHA3_256.new(bytes(m, "utf-8")).hexdigest()
    h1=int(h,16)%q
    b,c=0,0

    E=[]
    while b==0 or c==0 or xgcd(c,q)[0]!=1 :
        k=randint(1,q-1)
        B=ec_fast_mult(EC,G,k)
        b=B[0]%q
        c=((h1+sk*b)*pow(k,-1,q))%q
    #print(b!=0 and c!=0 and xgcd(c,q)[0]==1)
    return [b,c]

In [296]:
pk,sk=ecdsa_keygen(EC25519)
#il faut le mettre là parce que les fonctions sign ne sont pas deterministe
s=ecdsa_sign(EC25519,chaine_test,sk)
print(s)


[3331470594782130483011097429700286770270262463422966729550791958561059398053, 1291443876146189904261376286047217861580053947499864410309195455226119175386]


In [297]:
from Crypto.Hash import SHA3_256


def verif_signature(EC,m,s,pk):
    q=EC[-1]
    s_prim=s.copy()
    print(s_prim)
    b,c=s_prim
    b=b%q
    c=c%q
    G,q=EC[-2:].copy()
    p=EC[0]
    
    h = SHA3_256.new(bytes(m, "utf-8")).hexdigest()
    h1=int(h,16)%q

    #print("c=",c, "reste =",xgcd(c,q)[0])
    premiere_partie=(h1*pow(c,-1,q))%q
    deuxieme_partie=(b*pow(c,-1,q))%q

    P=ec_fast_mult(EC,G,premiere_partie)
    Q=ec_fast_mult(EC,pk,deuxieme_partie)

    R=ec_add(EC,P,Q)
    x_r=R[0]%q
    print(b)
    print(x_r)
    return print(x_r==b)



EC2=[]
verif_signature(EC25519,chaine_test,s,pk)


[3331470594782130483011097429700286770270262463422966729550791958561059398053, 1291443876146189904261376286047217861580053947499864410309195455226119175386]
3331470594782130483011097429700286770270262463422966729550791958561059398053
3741457929259307670362635121825871607967983665809989295470548534526698533774
False


In [298]:
def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1


In [299]:
a1=57896044618658097711785492504343953926634992332820282019728791901641727051837
b1=398341948620716521344
q1 = 2**252 + 27742317777372353535851937790883648493
G1=(2, 2587177637973221124604506650587198648324617568965701997790174145338249409797)
p1=(2**255-19)


EC25519=[p1,a1,b1,G1,q1]
print(isPrime(q1))


ec_fast_mult(EC25519,G1,q1)

True


'ZERO'

In [300]:
# pour ouvrir le fichier et stocker l’objet correspondant dans la variable f :
f = open("testfile.txt", 'r')
# pour stocker dans une chaine de caracteres s le contenu du fichier associe a f
s = f.read()
# pour fermer le fichier (ne pas oublier)
f.close()




h_long=sha3_256(bytes(chaine_test, 'utf-8')).hexdigest()

with open('publickey.txt', 'r') as fichier1:
    # Lire la première ligne du fichier
    premiere_ligne = fichier1.readline().strip()
    deuxieme_ligne=fichier1.readline().strip()

pk1=[premiere_ligne,deuxieme_ligne]


with open("rfc8031.txt", 'r') as f1:
    s = f1.read()



for i in range(0,100):
    if i<=9 :
        numero="0"+str(i)
    else :
        numero=str(i)
    #print("signature"+a+".txt")
    
    with open("signature"+numero+".txt", 'r') as fichier2:

    # Lire la première ligne du fichier
        premiere_ligne_b = fichier2.readline()
        deuxieme_ligne_c=fichier2.readline()

        signature=[int(premiere_ligne_b),int(deuxieme_ligne_c)]

    if verif_signature(EC25519,s,signature,pk1)==1:
        print("La signature",i,"est valide")
        break
    fichier2.close()

f1.close()

fichier1.close()





[2850083096520031701871339365433379873485852542900367142954126271251184431277, 3110873759805525005096085551910726233368612377829915059173389111083121554252]
2850083096520031701871339365433379873485852542900367142954126271251184431277
5580009742947789683104160701810121740999966994065697672095391217425441796786
False
[6255920760412435992725419631485432945916388104596630251165646504753771312814, 24845999883621851130476617749405115481200970810044357343994801454256788115]
6255920760412435992725419631485432945916388104596630251165646504753771312814
1795774073598507301597472318126743722758665640475325390803980710166911421760
False
[6248673897239664565433438514001553238996355626776840947656818620162230342787, 4142157798311313018116709268308828144990850050716614800318355176313259193874]
6248673897239664565433438514001553238996355626776840947656818620162230342787
2949236737187522612335761110203459411650679966383826972104084764066615835194
False
[11585487469600132697581358825723882542827952816753

1158548746960013269758135882572388254282795281675375255278865215009513249724
5528438154117715974804264793852120276812256505763064975237434920368583964839
False
[5294700890303901471700637738602941523756745911391777653661264939551936551367, 3845067417865485698292450996436100807201437318563666658368356699315062542128]
5294700890303901471700637738602941523756745911391777653661264939551936551367
1827986309768099563234964154272941731573140208508062139497706449672841422457
False
[4921856300579114565856073045531077071147387960950992022451740435887264577522, 4250208212394057450924528910168624047650177167132419807049400871510028529675]
4921856300579114565856073045531077071147387960950992022451740435887264577522
2176956453560334976286878984393235088537867913850700813125221233238100170217
False
[2027367506482871384300052244407750291425436782491375562393410865103221639727, 813628940931919587441430149523718406175693739848561477010106776828747467272]
20273675064828713843000522444077502914254367824913

In [301]:
1<<2
bin(31)
liste_1111=1
liste_1100_ver_1=7
for i in range(0,2):
        liste_1111=(liste_1111<<1)+1
liste_1111=int(bin(liste_1111),2)


print(bin(63))
liste_1111=liste_1111&63
print(bin(liste_1111))
liste_1100_ver_1=liste_1111-3
print(bin(liste_1100_ver_1))
print("000"+bin(liste_1111&liste_1100_ver_1))

0b111111
0b111
0b100
0000b100


In [302]:
import math


def random_bits(bit_length):
    # Générer un nombre aléatoire avec la taille de bits donnée
    random_number = random.getrandbits(bit_length)
    
    # Convertir le nombre aléatoire en chaîne binaire et la remplir avec des zéros à gauche si nécessaire
    binary_string = bin(random_number)[2:].zfill(bit_length)
    
    return int(binary_string,2)

def transformation_bit():
    random_i=random_bits(8*32)
    random_r=random_bits(8*32)
    #random_i,random_r=bin(random_i),bin(random_r)

    liste_1111_ver1,liste_1111_ver2=1,1

    random_i_1=int(bin(random_i)[0:9],2)
    random_i_2=int(bin(random_i)[-8:],2)
    
    taille_random_i_1=math.ceil(log2(random_i_1))
    taille_random_i_2=math.ceil(log2(random_i_2))
    for i in range(0,taille_random_i_1-3-1):
        liste_1111_ver1=(liste_1111_ver1<<1)+1
    
    for i in range(0,taille_random_i_2-3-1):
        liste_1111_ver2=(liste_1111_ver2<<1)+1


    taille_random_i=math.ceil(log2(random_i))
   
    print("taille 1111",math.ceil(log2(liste_1111_ver1)))
    print("taille de random_i_1",int(math.log2(random_i_1)) + 1)


    liste_1100_ver_1=liste_1111_ver1-3
    liste_000100_ver_1=(random_i_1&liste_1100_ver_1)

    liste_1100_ver_2=liste_1111_ver2-3
    liste_000100_ver_2=(random_i_2&liste_1100_ver_2)

    

    print("Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i \n",bin(liste_1111_ver1))
    print("Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin \n",bin(liste_1100_ver_1))
    #liste_000100_ver_1_ver_1_ver_1=liste_1100_ver_1&liste_1111
    print("Le nombre qu'on veut changer random_i_1 et sa taille :\n",bin(random_i_1),)
    print("Normalement on a mis les deux bits les plus significatifs à 0\n",bin(liste_000100_ver_1))
    print("Transition \n")
 
    print("taille 1111",math.ceil(log2(liste_1111_ver2)))
    print("taille de random_i_2",int(math.log2(random_i_2)) + 1)

    print("Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i \n",bin(liste_1111_ver2))
    print("Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin \n",bin(liste_1100_ver_2))
    #liste_000100_ver_1_ver_1_ver_1=liste_1100_ver_1&liste_1111
    print("Le nombre qu'on veut changer random_i_2 :\n",bin(random_i_2))
    print("Normalement on a mis les deux bits les plus significatifs à 0\n",bin(liste_000100_ver_2))

    fixed_i = liste_000100_ver_1 + int(bin(random_i)[9:-8],2) + liste_000100_ver_2

    liste_1111_ver1,liste_1111_ver2=1,1

    random_r_1=int(bin(random_r)[0:9],2)
    random_r_2=int(bin(random_r)[-8:],2)
    taille_random_r=math.ceil(log2(random_r))


    taille_random_r_1=math.ceil(log2(random_r_1))
    taille_random_r_2=math.ceil(log2(random_r_2))

    for i in range(0,taille_random_r_1-3-1):
        liste_1111_ver1=(liste_1111_ver1<<1)+1
    
    for i in range(0,taille_random_r_2-3-1):
        liste_1111_ver2=(liste_1111_ver2<<1)+1

    liste_1100_ver_1=liste_1111_ver1-3
    liste_000100_ver_1=(random_r_1&liste_1100_ver_1)
    liste_1100_ver_2=liste_1111_ver2-3
    liste_000100_ver_2=(random_r_2&liste_1111_ver2)&liste_1100_ver_2

    fixed_j=liste_000100_ver_1 + int(bin(random_r)[9:-8],2) +liste_000100_ver_2
    


    return[fixed_i,fixed_j]


transformation_bit()




taille 1111 4
taille de random_i_1 7
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b1111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b1100
Le nombre qu'on veut changer random_i_1 et sa taille :
 0b1111110
Normalement on a mis les deux bits les plus significatifs à 0
 0b1100
Transition 

taille 1111 0
taille de random_i_2 3
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b1
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 -0b10
Le nombre qu'on veut changer random_i_2 :
 0b100
Normalement on a mis les deux bits les plus significatifs à 0
 0b100


[3270423700199134716103650392259281909644537693967089633752827045491653356,
 760760289141303395240185455824249892403568576278856655030525171883411884]

In [303]:
fixed_i,fixed_r=transformation_bit()

def keygen_private(fixed_i,fixed_r,EC):
    d_i = fixed_i.to_bytes(32, byteorder='little')
    d_r= fixed_r.to_bytes(32, byteorder='little')
    pub_i=ec_fast_mult(EC25519,EC25519[-2],d_i) #ou calculer ec_fast_mult(G,donnees_encodee1) avec les données de la courbe
    pub_r=ec_fast_mult(EC25519,EC25519[-2],d_r)

    Shared_secret=ec_fast_mult(EC25519,pub_i,d_i)
    if Shared_secret==ec_fast_mult(EC25519,pub_r,d_r) :  #et la aussi on doit faire ec_fast_mult(G,donnees_encodee2)
        return Shared_secret  #PROBLEME: ça ne peut pas être la clé privée
    else:
        print("Le secret n'est pas commun")


taille 1111 4
taille de random_i_1 7
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b1111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b1100
Le nombre qu'on veut changer random_i_1 et sa taille :
 0b1111110
Normalement on a mis les deux bits les plus significatifs à 0
 0b1100
Transition 

taille 1111 4
taille de random_i_2 7
Normalement ça c'est un nombre 11111... qui  a deux bits en moins que random_i 
 0b1111
Normalement ça c'est le même nombre que 11100... qui a la même taille que en haut mais avec deux 0 a la fin 
 0b1100
Le nombre qu'on veut changer random_i_2 :
 0b1011001
Normalement on a mis les deux bits les plus significatifs à 0
 0b1000
